In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

d:\MUTUAL FILES\Book-recommender\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

books_data = pd.read_csv('./books_cleaned.csv')

In [5]:
books_data['tagged_description'].to_csv('books_descriptions.txt',
                                        index=False,
                                        header=False,
                                        sep='\n')

In [6]:
books_data

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web : A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5591,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5592,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5593,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that : Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [7]:
raw_document = TextLoader('books_descriptions.txt', encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0,
                                      chunk_overlap=0,
                                      separator='\n')
documents = text_splitter.split_documents(raw_document)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [8]:
documents[0]

Document(metadata={'source': 'books_descriptions.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [9]:
import os

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=os.getenv("api_key"))

In [10]:
db_books = Chroma.from_documents(documents, embedding=embeddings)


In [13]:
# Get the first embedding using the Chroma API
result = db_books.get(ids=[db_books._collection.get()["ids"][0]],
                      include=["embeddings"])
first_embedding = result["embeddings"][0]

print("First Document Embedding:")
print(first_embedding)


First Document Embedding:
[ 4.06641439e-02  1.51469223e-02 -2.25926284e-02 -2.04017460e-02
  9.11131501e-02  1.12647060e-02 -3.76385846e-03 -2.00768001e-02
  4.05141711e-02  4.99979418e-04  2.10418180e-02 -3.85735813e-03
  4.42180894e-02 -4.85407142e-03  1.44904014e-02  3.87674645e-02
  2.20110193e-02  3.40537466e-02  2.90618348e-03 -7.05178231e-02
  1.34904664e-02  2.20103357e-02 -6.30705804e-02 -1.20196417e-02
 -1.57226045e-02 -1.32665960e-02  2.97435131e-02 -1.11909837e-01
  1.73106268e-02 -3.14157945e-03 -2.12021302e-02  3.27339321e-02
 -3.35249268e-02  1.67867336e-02 -1.27489297e-02 -8.05761516e-02
  3.39178741e-02  8.90372135e-03  4.96251471e-02 -1.00362059e-02
  4.82562371e-02 -1.77595243e-02 -2.43768077e-02  7.53116980e-03
  2.56310999e-02 -6.04314841e-02 -1.00422837e-02  4.26634178e-02
  4.90172692e-02 -3.02027948e-02  1.43846059e-02  8.17488506e-03
  7.78097427e-03 -3.86324450e-02 -1.44275595e-02 -1.38131576e-02
  5.90928905e-02  5.22691407e-04 -9.82714072e-02  3.18928435e-02

In [15]:
query = "A book about the history of the Roman Empire"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='7d56b401-d43c-47ad-9327-b8a915d47b78', metadata={'source': 'books_descriptions.txt'}, page_content='9780679724773 The emperor Claudius tells of his life during the reigns of Augustus, Tiberius, and Caligula and the events that led to his rise to power in a classic novel reconstructing ancient Rome'),
 Document(id='e74e4e40-295b-4dac-8687-0f84bec3515b', metadata={'source': 'books_descriptions.txt'}, page_content='9780688093686 A story tracing the creation of Republican Rome presents those who founded an empire, including Marius and Sulla, each determined to become the First Man of Rome'),
 Document(id='b332b840-07ce-4e43-bbb3-e434fdd7c5f6', metadata={'source': 'books_descriptions.txt'}, page_content="9780380710829 The lives of ancient Rome's men--general Gaius Marius and his rival Lucius Cornelius Sulla--unfold amid Republican Rome's struggle in a world of treachery and barbarism"),
 Document(id='232b6a39-d823-4436-bb6a-e730e496d949', metadata={'source': 'books_description

In [ ]:
books_data[books_data['isbn13'] == int(docs[0].page_content.split()
                                       [0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3317,9780679724773,067972477X,"I, Claudius",Robert Graves,Fiction,http://books.google.com/books/content?id=cRzDT...,The emperor Claudius tells of his life during ...,1989.0,4.27,468.0,43469.0,"I, Claudius : From the Autobiography of Tiberi...",9780679724773 The emperor Claudius tells of hi...


In [19]:
def retrieve_semantic_recommendations(query: str,
                                      top_k: int = 10) -> pd.DataFrame:
    records = db_books.similarity_search(query, k=50)

    book_list = []
    for i in range(0, len(records)):
        book_list += [int(records[i].page_content.strip('"').split()[0])]
    return books_data[books_data['isbn13'].isin(book_list)].head(top_k)


In [22]:
retrieve_semantic_recommendations("A book about cooking", )

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
111,9780060277406,0060277408,The Secret Garden Cookbook,Amy Cotler,Juvenile Nonfiction,http://books.google.com/books/content?id=c7E_H...,Frances Hodgson Burnett's The Secret Garden de...,1999.0,4.28,128.0,142.0,The Secret Garden Cookbook : Recipes Inspired ...,9780060277406 Frances Hodgson Burnett's The Se...
283,9780060899226,0060899220,Kitchen Confidential Updated Ed,Anthony Bourdain,Biography & Autobiography,http://books.google.com/books/content?id=bxLbZ...,"A deliciously funny, delectably shocking banqu...",2007.0,4.06,312.0,168706.0,Kitchen Confidential Updated Ed : Adventures i...,"9780060899226 A deliciously funny, delectably ..."
755,9780141010373,0141010371,Jamie's Kitchen,Jamie Oliver,Cookery,http://books.google.com/books/content?id=WG3SN...,The No 1 bestseller I love this book. It's the...,2004.0,3.89,336.0,4012.0,Jamie's Kitchen,9780141010373 The No 1 bestseller I love this ...
824,9780142000212,0142000213,Bridget Jones's Guide to Life,Helen Fielding,Fiction,http://books.google.com/books/content?id=jJLdC...,"Bridget Jones, the main character in the novel...",2001.0,3.48,64.0,1083.0,Bridget Jones's Guide to Life,"9780142000212 Bridget Jones, the main characte..."
929,9780152057084,0152057080,Apple Pie Fourth of July,Janet S. Wong;Margaret Chodos-Irvine,Juvenile Fiction,http://books.google.com/books/content?id=am2MM...,A Chinese American child fears that the food h...,2006.0,3.71,40.0,298.0,Apple Pie Fourth of July,9780152057084 A Chinese American child fears t...
1198,9780307238276,030723827X,Giada's Family Dinners,Giada De Laurentiis,Cooking,http://books.google.com/books/content?id=lfRvD...,The Food Network host and best-selling author ...,2006.0,3.96,256.0,13866.0,Giada's Family Dinners,9780307238276 The Food Network host and best-s...
1217,9780307346582,0307346587,Everyday Pasta,Giada De Laurentiis,Cooking,http://books.google.com/books/content?id=oMiU2...,Collects Italian cookery recipes that range fr...,2007.0,4.09,240.0,8719.0,Everyday Pasta : Favorite Pasta Recipes for Ev...,9780307346582 Collects Italian cookery recipes...
1410,9780316013260,0316013269,Julie and Julia,Julie Powell,Biography & Autobiography,http://books.google.com/books/content?id=KdoZH...,Now in paperback-the format in which it's dest...,2006.0,3.70,307.0,2229.0,Julie and Julia : My Year of Cooking Dangerously,9780316013260 Now in paperback-the format in w...
1763,9780375509179,0375509178,Shakespeare's Kitchen,Francine Segan,Cooking,http://books.google.com/books/content?id=ecwAA...,A noted food historian brings to life the clas...,2003.0,3.99,288.0,108.0,Shakespeare's Kitchen : Renaissance Recipes fo...,9780375509179 A noted food historian brings to...
1824,9780375758737,0375758739,Comfort Me With Apples,Ruth Reichl,Biography & Autobiography,http://books.google.com/books/content?id=keNpY...,The noted food critic describes her odyssey fr...,2002.0,4.03,302.0,17418.0,Comfort Me With Apples : More Adventures at th...,9780375758737 The noted food critic describes ...
